In [ ]:
!pip uninstall -y transformers
!pip install transformers==4.44.2 torch>=2.0.0 torchvision>=0.15.0 ultralytics>=8.0.0 opencv-python>=4.8.0 numpy>=1.24.0 pillow>=9.0.0 accelerate>=0.20.0 huggingface-hub

In [ ]:
from huggingface_hub import login
login(token="insert-token")  # replace with actual hf token

In [ ]:
!mkdir -p human-centric-video-stabilization/src

In [ ]:
%%writefile human-centric-video-stabilization/requirements.txt
torch>=2.0.0
torchvision>=0.15.0
transformers==4.44.2
ultralytics>=8.0.0
opencv-python>=4.8.0
numpy>=1.24.0
pillow>=9.0.0
accelerate>=0.20.0
huggingface-hub
scipy>=1.10.0

In [ ]:
%%writefile human-centric-video-stabilization/src/__init__.py
# Empty file to make src a package

In [ ]:
%%writefile human-centric-video-stabilization/src/background.py
import cv2
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from transformers import AutoModelForImageSegmentation
import warnings
warnings.filterwarnings("ignore")  # suppress minor warnings

def load_background_remover(device='cpu'):
    """
    loading the RMBG-2.0 model for background removal.
    requires Hugging Face authentication for gated model.
    returns: model, transform_image
    """
    # setting precision for efficiency (as per model docs)
    torch.set_float32_matmul_precision('high')

    # loading model WITHOUT dtype (causes TypeError in BiRefNet.__init__)
    model = AutoModelForImageSegmentation.from_pretrained(
        'briaai/RMBG-2.0',
        trust_remote_code=True
    )

    # moving to device (keeping float32 on GPU for compatibility)
    model = model.to(device)

    model.eval()

    # image size for model input (as per model docs)
    image_size = (1024, 1024)

    # preprocessing transform (ImageNet normalization, as per model docs)
    transform_image = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    return model, transform_image

def remove_background(frame, model, transform_image):
    """
    remove background from a BGR frame using RMBG-2.0.
    returns RGBA numpy array (BGR + Alpha), where alpha is soft (0-255) from model probability.
    """
    if frame is None or frame.size == 0:
        raise ValueError("Invalid frame input")

    device = next(model.parameters()).device

    # converting BGR to RGB for model
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_rgb = Image.fromarray(rgb)
    original_size = pil_rgb.size  # (W, H)

    # preprocessing
    input_tensor = transform_image(pil_rgb).unsqueeze(0).to(device)

    # inference (as per model docs: model(input)[-1].sigmoid())
    with torch.no_grad():
        pred = model(input_tensor)[-1].sigmoid().cpu()[0].squeeze()  # (H=1024, W=1024), 0-1 prob

    # post-processing: Convert to PIL grayscale (0-255), resize to original (LANCZOS for quality)
    pred_pil = transforms.ToPILImage()(pred)
    mask_pil = pred_pil.resize(original_size, Image.Resampling.LANCZOS)

    # alpha mask as uint8
    alpha = np.array(mask_pil)  # (H, W), 0-255

    # stacking BGR + Alpha
    rgba = np.dstack([frame, alpha])

    return rgba

In [ ]:
# using only high-confidence keypoints)
%%writefile human-centric-video-stabilization/src/pose.py
import numpy as np
from ultralytics import YOLO

def load_pose_model(model_path='yolov8n-pose.pt', device='cpu'):
    """
    loading YOLOv8-Pose model.
    """
    model = YOLO(model_path)
    model.to(device)  # moving to device always (works for CPU too)
    return model

def detect_pose_center(model, frame):
    results = model.track(frame, persist=True, verbose=False)  # enabling tracking
    if len(results) == 0 or results[0].keypoints is None:
        return None, []

    # assuming main person is the one with ID 0 or lowest ID; filter if multi-person
    keypoints = results[0].keypoints
    if len(keypoints.xy) == 0:
        return None, []

    xy = keypoints.xy[0].cpu().numpy()
    conf = keypoints.conf[0].cpu().numpy() if keypoints.conf is not None else np.ones(17)
    keypoints_data = [(float(xy[i][0]), float(xy[i][1]), float(conf[i])) for i in range(17)]

    # using average of high-confidence hips and shoulders (indices: 5/6 shoulders, 11/12 hips)
    indices = [5, 6, 11, 12]
    points = xy[indices]
    confs = conf[indices]
    valid_points = points[confs > 0.5]
    if len(valid_points) > 0:
        center_x = int(np.mean(valid_points[:, 0]))
        center_y = int(np.mean(valid_points[:, 1]))
        return (center_x, center_y), keypoints_data
    return None, keypoints_data

In [ ]:
%%writefile human-centric-video-stabilization/src/stabilization.py
import numpy as np
from scipy.linalg import block_diag  # for multi-dimensional Kalman

class KalmanFilter:
    def __init__(self, process_noise=0.1, measurement_noise=1.0):
        # state: [x, y, vx, vy] (position + velocity)
        self.state = np.zeros(4)  # initial state
        self.P = np.eye(4) * 1e-3  # covariance
        self.Q = block_diag(process_noise * np.eye(2), process_noise * np.eye(2))  # processing noise
        self.R = measurement_noise * np.eye(2)  # measuring noise
        self.F = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]])  # transition
        self.H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])  # measurement

    def update(self, measurement):
        if measurement is None:
            # predicting only if no measurement
            self.state = self.F @ self.state
            self.P = self.F @ self.P @ self.F.T + self.Q
            return self.state[:2]

        # predict
        self.state = self.F @ self.state
        self.P = self.F @ self.P @ self.F.T + self.Q

        # update
        y = measurement - self.H @ self.state
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.state += K @ y
        self.P = (np.eye(4) - K @ self.H) @ self.P
        return self.state[:2]

def compute_transform(center_point, target_point, kalman):
    if center_point is not None:
        center_point = np.array(center_point, dtype=np.float32)
    smoothed_center = kalman.update(center_point)
    dx = target_point[0] - smoothed_center[0]
    dy = target_point[1] - smoothed_center[1]
    return np.array([dx, dy], dtype=np.float32)  # Kalman handles smoothing

In [ ]:
# (adding border handling in warping to avoid black edges; transform keypoints for correct visualization on stabilized frame)
%%writefile human-centric-video-stabilization/src/rendering.py
import cv2
import numpy as np
import json
from pathlib import Path

def create_stabilized_frame(frame, foreground_rgba, current_transform, blur_bg=False, blur_kernel=15, visualize=False, keypoints_data=[], target_point=(0,0)):
    """
    create stabilized frame:
    - warp original frame by inverse transform (background drifts).
    - warp foreground by forward transform (subject fixed).
    - alpha blend.
    returns: stabilized BGR frame.
    """
    h, w = frame.shape[:2]

    # translation matrices
    tx, ty = current_transform
    M_forward = np.float32([[1, 0, tx], [0, 1, ty]])
    M_inverse = np.float32([[1, 0, -tx], [0, 1, -ty]])

    # warping with border handling
    warped_original = cv2.warpAffine(frame, M_inverse, (w, h), borderMode=cv2.BORDER_REPLICATE)
    if blur_bg:
        warped_original = cv2.GaussianBlur(warped_original, (blur_kernel, blur_kernel), 0)

    # warping foreground RGBA (forward: subject to target) with transparent border
    stabilized_fg_rgba = cv2.warpAffine(foreground_rgba, M_forward, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=[0, 0, 0, 0])

    # extracting for blending
    alpha = stabilized_fg_rgba[:, :, 3].astype(np.float32) / 255.0
    fg_bgr = stabilized_fg_rgba[:, :, 0:3].astype(np.float32)
    bg_bgr = warped_original.astype(np.float32)

    # soft alpha blending
    blended = (1 - alpha[:, :, np.newaxis]) * bg_bgr + alpha[:, :, np.newaxis] * fg_bgr
    stabilized_frame = np.clip(blended, 0, 255).astype(np.uint8)

    if visualize:
        cv2.circle(stabilized_frame, target_point, 10, (0, 255, 0), 2)  # target marker
        for kp in keypoints_data:  # from pose detection
            if kp[2] > 0.5:  # conf threshold
                # applying forward transform to keypoints for correct position on stabilized frame
                new_x = kp[0] + tx
                new_y = kp[1] + ty
                if 0 <= new_x < w and 0 <= new_y < h:  # clip to frame
                    cv2.circle(stabilized_frame, (int(new_x), int(new_y)), 5, (255, 0, 0), -1)

    return stabilized_frame

def save_pose_data(poses_list, output_path):
    """
    saving pose keypoints to JSON.
    poses_list: list of {'frame': int, 'keypoints': list of [x, y, conf]}
    """
    with open(output_path, 'w') as f:
        json.dump(poses_list, f, indent=2)

In [ ]:
%%writefile human-centric-video-stabilization/src/run.py
#!/usr/bin/env python3
"""
CLI entry point for human-centric video stabilization.
usage: python src/run.py --input /path/to/video.mp4 [options]
"""
import argparse
import cv2
import numpy as np
import os
import time
from pathlib import Path
import torch  # for device check
from background import load_background_remover, remove_background
from pose import load_pose_model, detect_pose_center
from stabilization import compute_transform, KalmanFilter
from rendering import create_stabilized_frame, save_pose_data

def process_video(input_path, output_stab="stabilized.mp4", output_comp="comparison.mp4",
                  poses_path="poses.json", target_x=None, target_y=None,
                  device='cpu', blur_bg=False, visualize=False):
    """
    Main processing pipeline.
    """
    # detecting device if 'auto'
    if device == 'auto':
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Auto-detected device: {device}")

    # loading models
    print("Loading models...")
    model_bg, transform_bg = load_background_remover(device)
    model_pose = load_pose_model(device=device)

    # setup video
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {input_path}")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    target_point = (target_x or width // 2, target_y or height // 2)

    # video writers
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_stab = cv2.VideoWriter(output_stab, fourcc, fps, (width, height))
    out_comp = cv2.VideoWriter(output_comp, fourcc, fps, (width * 2, height))

    # init
    kalman = KalmanFilter(process_noise=0.05, measurement_noise=0.5)  # tune as wish (hyperparam)
    poses = []
    frame_count = 0
    start_time = time.time()
    print(f"Target point: {target_point}")
    print(f"Using device: {device}")
    print("Processing frames...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # detecting pose and center
        center_point, keypoints_data = detect_pose_center(model_pose, frame)

        # computing transform
        current_transform = compute_transform(center_point, target_point, kalman)

        # removing background
        foreground_rgba = remove_background(frame, model_bg, transform_bg)

        # stabilizing and composite
        stabilized_frame = create_stabilized_frame(frame, foreground_rgba, current_transform, blur_bg=blur_bg, visualize=visualize, keypoints_data=keypoints_data, target_point=target_point)

        # saving pose data
        poses.append({'frame': frame_count, 'keypoints': keypoints_data, 'transform': current_transform.tolist()})

        # labels for comparison
        orig_labeled = frame.copy()
        stab_labeled = stabilized_frame.copy()
        cv2.putText(orig_labeled, "Original", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
        cv2.putText(stab_labeled, "Stabilized", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

        comp_frame = np.hstack([orig_labeled, stab_labeled])

        # write
        out_stab.write(stabilized_frame)
        out_comp.write(comp_frame)

        frame_count += 1
        if frame_count % 10 == 0:  # frequent updates for slow processing
            print(f"Processed {frame_count} frames...")

    # cleanup
    cap.release()
    out_stab.release()
    out_comp.release()

    # saving poses
    save_pose_data(poses, poses_path)

    # runtime
    elapsed = time.time() - start_time
    actual_fps = frame_count / elapsed if elapsed > 0 else 0
    print(f"✅ Complete! {frame_count} frames processed in {elapsed:.2f}s ({actual_fps:.2f} FPS).")
    print(f"Stabilized: {output_stab}")
    print(f"Comparison: {output_comp}")
    print(f"Poses: {poses_path}")

def main():
    parser = argparse.ArgumentParser(description="Human-Centric Video Stabilization")
    parser.add_argument('--input', '-i', required=True, help="Input video path")
    parser.add_argument('--output-stab', '-s', default="stabilized.mp4", help="Stabilized output path")
    parser.add_argument('--output-comp', '-c', default="comparison.mp4", help="Comparison output path")
    parser.add_argument('--poses', '-p', default="poses.json", help="Pose data output path")
    parser.add_argument('--target-x', type=int, help="Target X position (default: width/2)")
    parser.add_argument('--target-y', type=int, help="Target Y position (default: height/2)")
    parser.add_argument('--device', choices=['cpu', 'cuda', 'auto'], default='auto', help="Device (auto detects GPU if available)")
    parser.add_argument('--blur-bg', action='store_true', help="Blur background for emphasis")
    parser.add_argument('--visualize', action='store_true', help="Visualize keypoints and target point")

    args = parser.parse_args()

    if not os.path.exists(args.input):
        raise FileNotFoundError(f"Input video not found: {args.input}")

    process_video(
        input_path=args.input,
        output_stab=args.output_stab,
        output_comp=args.output_comp,
        poses_path=args.poses,
        target_x=args.target_x,
        target_y=args.target_y,
        device=args.device,
        blur_bg=args.blur_bg,
        visualize=args.visualize
    )

if __name__ == "__main__":
    main()

In [ ]:
!cd human-centric-video-stabilization && python src/run.py --input /content/8296233-hd_1080_1920_25fps.mp4 --output-stab /content/stabilized.mp4 --output-comp /content/comparison.mp4 --poses /content/poses.json --device auto --blur-bg --visualize